<a href="https://colab.research.google.com/github/Sirmj-1986/Sample-Angle-Mapper-Code/blob/main/CKI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install some packages
!pip install rasterio
!pip install earthpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.9 MB/s eta 0:00:00


In [2]:
# Import libraries
import pandas as pd
import numpy as np
import rasterio
import earthpy.plot as ep
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from matplotlib.colors import from_levels_and_colors
import seaborn as sns
import os

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
# Define path that contains the datasets
Sample_Path = '/content/drive/MyDrive/GEE_Exports/Training_Samples_Fixed.csv'
Image_Path = '/content/drive/MyDrive/GEE_Exports/ImageG_Stacked_Float.tif'


In [ ]:
# Define target and predictor variables
Bands = ['PC2', 'PC3', 'PC5', 'geology','B09', 'B08', 'B07', 'B05', 'B06', 'B3N',
  'Calcite_SAM', 'Kaolinite_SAM', 'IronOxide_SAM']  # Feature columns
LC = ['class']
Classes = [0, 1, 2, 3]
N_Classes = 4
Names = ["No_Alteration", "Calcite", "Kaolinite", "IronOxide"]
Palette = [
    '#008000',  # Green  for class 0 (No_Alteration)
    '#FF0000',  # Red    for class 1 (Calcite)
    '#0000FF',  # Blue   for class 2 (Kaolinite)
    '#FFD700',  # Gold   for class 3 (IronOxide)
]

In [ ]:
image = rasterio.open(Image_Path)

band_count = image.count
height = image.height
width = image.width
crs = image.crs
transform = image.transform

print(f"Image has {band_count} bands.")
print(f"Image dimensions: {height} rows x {width} columns")
print(f"Coordinate Reference System (CRS): {crs}")

band_indices_for_vis = [Bands.index('Calcite_SAM') + 1, Bands.index('Kaolinite_SAM') + 1, Bands.index('IronOxide_SAM') + 1]

image_vis = []
for b_index in band_indices_for_vis:
    image_vis.append(image.read(b_index))
image_vis = np.stack(image_vis)

plt.figure(figsize=(8,8))
ep.plot_rgb(
    image_vis,
    stretch=True
)
plt.show()

In [ ]:
# Load training data as a DataFrame
df = pd.read_csv(Sample_Path)

# Inspect first few rows
print(df.head())

# Separate features (X) and label (y)
X = df[Bands]
y = df['class']

# Ensure no missing values
print(f"Missing values in features: {X.isnull().sum().sum()}")
print(f"Missing values in label: {y.isnull().sum()}")

# Split into training and testing subsets
# (you can also do cross-validation if you prefer)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")


In [ ]:
# Combine X_train with y_train for easy plotting
df_train = X_train.copy()
df_train['class'] = y_train.values

# Set up the figure size
plt.figure(figsize=(10, 6))

# Create a count plot of 'class' to show the distribution of training samples for each class
sns.countplot(
    x='class',        # The column on the x-axis
    data=df_train,    # The DataFrame containing our features and labels
    palette=Palette   # Use the predefined color palette for each class
)

# Add a title to the plot
plt.title('Distribution of Samples Across Classes')

# Label the x-axis
plt.xlabel('Class')

# Label the y-axis
plt.ylabel('Number of Samples')

# Customize the x-ticks to show class names instead of numbers
plt.xticks(
    ticks=[0, 1, 2, 3],                           # Positions for each class
    labels=["No_Alteration", "Calcite", "Kaolinite", "IronOxide"]
)

# Display the plot
plt.show()

In [ ]:
# Combine X_train with y_train for easy plotting
df_train = X_train.copy()
df_train['class'] = y_train.values

# Scatter plot of B4 vs B8, colored by 'class'
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x='PC2',
    y='PC3',
    hue='class',
    data=df_train,
    palette=Palette
)
plt.title('Scatter Plot of PC2 vs PC3')
plt.xlabel('PC2 reflectance')
plt.ylabel('PC3 reflectance')
plt.legend(title='Class')
plt.show()


In [ ]:
g = sns.pairplot(df_train, vars=Bands, hue='class', palette=Palette)
g.fig.suptitle('Pairwise Scatter Plots for All Bands', y=1.02)

for ax in g.axes.flat:
    ax.set_xlabel(ax.get_xlabel(), fontsize=17, fontweight='bold')
    ax.set_ylabel(ax.get_ylabel(), fontsize=17, fontweight='bold')
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)

plt.show()

In [ ]:
# Define the export path
export_folder = '/content/drive/MyDrive/AGU'
export_filename = 'Pairwise_Scatter_Plot222.png'
export_path = os.path.join(export_folder, export_filename)

if not os.path.exists(export_folder):
    os.makedirs(export_folder)
g.savefig(export_path, dpi=1000, bbox_inches='tight')

print(f"Pair plot exported successfully to: {export_path}")

In [ ]:
# Libraries needed
from mpl_toolkits.mplot3d import Axes3D  # needed for 3D projections
from matplotlib.patches import Patch

# Create a color map from class labels to colors in the palette
color_map = {class_label: Palette[i] for i, class_label in enumerate(Classes)}

# Map each sample's class label to its corresponding color
colors = df_train['class'].map(color_map)

# Initialize the figure and 3D axis
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot:
#   x-axis => PC2 (Blue)
#   y-axis => PC3 (Red)
#   z-axis => PC5 (NIR)
scatter = ax.scatter(
    df_train['PC2'],
    df_train['PC3'],
    df_train['PC5'],
    c=colors,
    s=15  # point size (optional)
)

# Label the axes
ax.set_xlabel('PC2)', labelpad=15)
ax.set_ylabel('PC3', labelpad=15)
ax.set_zlabel('PC5', labelpad=15)

# Adjust distance of the "camera" to the plot (optional)
ax.dist = 11

# (Optional) Adjust the viewing angle for better orientation
# ax.view_init(elev=20, azim=30)  # e.g., 20° above, 30° rotation

# Create a custom legend
legend_elements = [
    Patch(facecolor=Palette[i], edgecolor='k', label=f'Class {Classes[i]}')
    for i in range(len(Classes))
]
ax.legend(
    handles=legend_elements,
    title="Classes",
    loc='upper left',
    bbox_to_anchor=(1.05, 1)  # move legend outside the plot if you want
)

# Add a title
plt.title('3D Scatter Plot of Blue, Red, and NIR Bands', pad=20)

plt.show()



In [ ]:
# Import TSNE
from sklearn.manifold import TSNE

# 1. Initialize the t-SNE model
tsne = TSNE(
    n_components=2,  # Project down to 2D
    perplexity=30,   # Typical range is 5-50; tune as needed
    random_state=42, # For reproducibility
    n_iter=1000      # Number of gradient descent iterations
)

# 2. Fit and transform your feature matrix
X_tsne = tsne.fit_transform(X)

# 3. Plot the results
plt.figure(figsize=(10, 8))

# color_map = {class_label: Palette[i] for i, class_label in enumerate(Classes)}
colors = [color_map[label] for label in y]

plt.scatter(
    X_tsne[:, 0],  # t-SNE x-coordinates
    X_tsne[:, 1],  # t-SNE y-coordinates
    c=colors,
    s=10,          # Marker size
    alpha=0.7      # Transparency
)

plt.title("t-SNE Projection of Land Cover Samples")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")

# (Optional) Add a legend
# If you have a legend, you can manually create it or use existing patches
# or you can skip the legend if it becomes cluttered in high-dimensional data

plt.show()

In [ ]:
# 1. Melt the DataFrame into a "long" format for box plotting
df_melt = df_train.melt(
    id_vars='class',               # Keep 'class' as an identifier
    value_vars=Bands,              # The band columns to melt
    var_name='Band',               # Name for the new "band" column
    value_name='Reflectance'       # Name for the new "reflectance" column
)

# 2. Create a box plot
plt.figure(figsize=(12, 8))
sns.boxplot(
    x='Band',
    y='Reflectance',
    hue='class',
    data=df_melt,
    palette=Palette                # (Optional) use the predefined color palette
)

# 3. Customize the plot
plt.title("Box Plots of Reflectance Values by Band and Class")
plt.xlabel("Spectral Band")
plt.ylabel("Reflectance Value")
plt.legend(title="Class", loc="upper right")

# 4. Display the plot
plt.show()


In [ ]:
# Define hyperparameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Define hyperparameter grid for SVM
param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Define hyperparameter grid for XGBoost
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

print("Hyperparameter grids defined.")

In [ ]:
# Random Forest Grid Search
rf_grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid_rf,
    cv=5
)
rf_grid_search.fit(X_train, y_train)

# SVM Grid Search
svm_grid_search = GridSearchCV(
    SVC(random_state=42, probability=True), # Add probability=True
    param_grid_svm,
    cv=5
)
svm_grid_search.fit(X_train, y_train)

# XGBoost Grid Search
from xgboost import XGBClassifier # Import XGBClassifier

xgb_grid_search = GridSearchCV(
    XGBClassifier(
        objective='multi:softmax',
        num_class=N_Classes,
        eval_metric='mlogloss',
        random_state=42
    ),
    param_grid_xgb,
    cv=5
)
xgb_grid_search.fit(X_train, y_train)

print("Grid search for all models complete.")

In [ ]:
# Get the best models from grid search
best_rf = rf_grid_search.best_estimator_
best_svm = svm_grid_search.best_estimator_
best_xgb = xgb_grid_search.best_estimator_

# Make predictions on the test set using the best models
rf_preds = best_rf.predict(X_test)
svm_preds = best_svm.predict(X_test)
xgb_preds = best_xgb.predict(X_test)

# Evaluate each model
models = {
    "Random Forest": rf_preds,
    "SVM": svm_preds,
    "XGBoost": xgb_preds
}

for name, preds in models.items():
    print(f"\n** {name} **")
    print("Accuracy:", (preds == y_test).mean())

    # Classification Report
    print(classification_report(y_test, preds))

    # Confusion Matrix
    cm = confusion_matrix(y_test, preds, labels=Classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=Classes)
    disp.plot(cmap='Blues')
    plt.title(f"{name} - Confusion Matrix")
    plt.show()

In [ ]:
# Read all bands as a NumPy array
# shape = (band_count, height, width)
img_array = image.read()

# For example, if the image bands match the ordering of 'Bands' exactly,
# index them accordingly. Make sure the array order aligns with your CSV.
print("Image array shape:", img_array.shape)  # Should be (#bands, height, width)

In [ ]:
# Transpose and reshape to [#pixels, #bands]
# If img_array is [bands, rows, cols]:
img_reshaped = img_array.reshape(band_count, -1).T  # shape => (rows*cols, bands)
print("Reshaped array for prediction:", img_reshaped.shape)


In [ ]:
# Make predictions on the entire image using each trained model
rf_prediction = best_rf.predict(img_reshaped)
svm_prediction = best_svm.predict(img_reshaped)
xgb_prediction = best_xgb.predict(img_reshaped)

print("Predictions made for all models on the image data.")

In [ ]:
# 1. Make predictions on the test set
rf_preds_tuned = best_rf.predict(X_test)
svm_preds_tuned = best_svm.predict(X_test)
xgb_preds_tuned = best_xgb.predict(X_test)

# 2. Create a dictionary of tuned models and their predictions
tuned_models = {
    "Tuned Random Forest": rf_preds_tuned,
    "Tuned SVM": svm_preds_tuned,
    "Tuned XGBoost": xgb_preds_tuned
}

# 3. Iterate through the tuned models and evaluate performance
for name, preds in tuned_models.items():
    print(f"\n** {name} **")
    print("Accuracy:", (preds == y_test).mean())

    # Classification Report
    print(classification_report(y_test, preds, target_names=Names))

    # Confusion Matrix
    cm = confusion_matrix(y_test, preds, labels=Classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=Classes)
    disp.plot(cmap='Blues')
    plt.title(f"{name} - Confusion Matrix")
    plt.show()

In [ ]:
from matplotlib.patches import Patch
import numpy as np # Ensure numpy is imported
import matplotlib.pyplot as plt # Ensure matplotlib.pyplot is imported
import rasterio.coords # Import rasterio.coords
import os # Import os for path joining

# Define the export folder
export_folder = '/content/drive/MyDrive/AGU'

# Create the export folder if it doesn't exist
if not os.path.exists(export_folder):
    os.makedirs(export_folder)

# Prepare a discrete colormap
# We need one more level than classes for from_levels_and_colors
levels = Classes + [max(Classes) + 1]
cmap, norm = from_levels_and_colors(levels, Palette)

# Reshape predictions back to image dimensions
rf_prediction_map = rf_prediction.reshape(height, width).astype(np.uint8)
svm_prediction_map = svm_prediction.reshape(height, width).astype(np.uint8)
xgb_prediction_map = xgb_prediction.reshape(height, width).astype(np.uint8)

# Calculate extent using rasterio.coords.BoundingBox
bbox = rasterio.coords.BoundingBox(
    left=transform.c,
    bottom=transform.f + transform.e * height,
    right=transform.c + transform.a * width,
    top=transform.f
)
extent = [bbox.left, bbox.right, bbox.bottom, bbox.top]

# Visualize Random Forest prediction map
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(rf_prediction_map, cmap=cmap, norm=norm, extent=extent, origin='upper')
ax.set_title("Random Forest Classification Map", fontsize=12)
ax.set_xlabel("Longitude", fontsize=12)
ax.set_ylabel("Latitude", fontsize=12)
cbar = fig.colorbar(im, ax=ax, shrink=0.7, orientation='horizontal', pad=0.1) # Add pad for spacing
tick_positions = [i + 0.5 for i in Classes]
cbar.set_ticks(tick_positions)
cbar.set_ticklabels(Names)

# Define export path and save the figure
export_filename_rf = 'RandomForest_Classification_Map.png'
export_path_rf = os.path.join(export_folder, export_filename_rf)
plt.savefig(export_path_rf, dpi=1000, bbox_inches='tight')
print(f"Random Forest classification map exported successfully to: {export_path_rf}")

plt.show()

# Visualize SVM prediction map
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(svm_prediction_map, cmap=cmap, norm=norm, extent=extent, origin='upper')
ax.set_title("SVM Classification Map", fontsize=12)
ax.set_xlabel("Longitude", fontsize=12)
ax.set_ylabel("Latitude", fontsize=12)
cbar = fig.colorbar(im, ax=ax, shrink=0.7, orientation='horizontal', pad=0.1) # Add pad for spacing
tick_positions = [i + 0.5 for i in Classes]
cbar.set_ticks(tick_positions)
cbar.set_ticklabels(Names)

# Define export path and save the figure
export_filename_svm = 'SVM_Classification_Map.png'
export_path_svm = os.path.join(export_folder, export_filename_svm)
plt.savefig(export_path_svm, dpi=1000, bbox_inches='tight')
print(f"SVM classification map exported successfully to: {export_path_svm}")

plt.show()

# Visualize XGBoost prediction map
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(xgb_prediction_map, cmap=cmap, norm=norm, extent=extent, origin='upper')
ax.set_title("XGBoost Classification Map", fontsize=12)
ax.set_xlabel("Longitude", fontsize=12)
ax.set_ylabel("Latitude", fontsize=12)
cbar = fig.colorbar(im, ax=ax, shrink=0.7, orientation='horizontal', pad=0.1) # Add pad for spacing
tick_positions = [i + 0.5 for i in Classes]
cbar.set_ticks(tick_positions)
cbar.set_ticklabels(Names)

# Define export path and save the figure
export_filename_xgb = 'XGBoost_Classification_Map.png'
export_path_xgb = os.path.join(export_folder, export_filename_xgb)
plt.savefig(export_path_xgb, dpi=1000, bbox_inches='tight')
print(f"XGBoost classification map exported successfully to: {export_path_xgb}")

plt.show()

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import from_levels_and_colors

# Define the path to the hillshade image
Hillshade_Path = '/content/drive/MyDrive/GEE_Exports/Hillshade.tif'

# Load the hillshade image
try:
    with rasterio.open(Hillshade_Path) as hillshade_image:
        hillshade_array = hillshade_image.read(1) # Read the first band (assuming it's a single-band hillshade)
        hillshade_extent = [hillshade_image.bounds.left, hillshade_image.bounds.right,
                          hillshade_image.bounds.bottom, hillshade_image.bounds.top]
except rasterio.errors.RasterioIOError:
    print(f"Error: Could not open or read the hillshade image at {Hillshade_Path}")
    hillshade_array = None

if 'extent' not in locals():

    bbox = rasterio.coords.BoundingBox(
        left=transform.c,
        bottom=transform.f + transform.e * height,
        right=transform.c + transform.a * width,
        top=transform.f
    )
    extent = [bbox.left, bbox.right, bbox.bottom, bbox.top]

try:
    rf_prediction_map = rf_prediction.reshape(height, width).astype(np.uint8)
    svm_prediction_map = svm_prediction.reshape(height, width).astype(np.uint8)
    xgb_prediction_map = xgb_prediction.reshape(height, width).astype(np.uint8)
except NameError:
    print("Error: Prediction maps (rf_prediction, svm_prediction, xgb_prediction) are not defined.")
    print("Please ensure the previous cells generating these predictions have been run.")
    rf_prediction_map = None
    svm_prediction_map = None
    xgb_prediction_map = None


if hillshade_array is not None and rf_prediction_map is not None:
    remaining_classes = [c for c in Classes if c != 0]
    remaining_names = [Names[i] for i in remaining_classes]
    remaining_palette = [Palette[i] for i in remaining_classes]

    levels_remaining = remaining_classes + [max(remaining_classes) + 1]
    cmap_remaining, norm_remaining = from_levels_and_colors(levels_remaining, remaining_palette)

    mask_rf_no_alteration = (rf_prediction_map == 0)
    mask_svm_no_alteration = (svm_prediction_map == 0)
    mask_xgb_no_alteration = (xgb_prediction_map == 0)

    rf_altered_map = rf_prediction_map.astype(float) # Convert to float to allow NaN
    rf_altered_map[mask_rf_no_alteration] = np.nan

    svm_altered_map = svm_prediction_map.astype(float)
    svm_altered_map[mask_svm_no_alteration] = np.nan

    xgb_altered_map = xgb_prediction_map.astype(float)
    xgb_altered_map[mask_xgb_no_alteration] = np.nan


    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.imshow(hillshade_array, cmap='gray', extent=hillshade_extent, origin='upper') # Plot hillshade first
    im = ax.imshow(rf_altered_map, cmap=cmap_remaining, norm=norm_remaining, extent=extent, origin='upper', alpha=0.7) # Overlay altered map with transparency
    ax.set_title("Random Forest Altered Areas on Hillshade", fontsize=12)
    ax.set_xlabel("Longitude", fontsize=12)
    ax.set_ylabel("Latitude", fontsize=12)

    # Create a custom legend for the remaining classes
    legend_elements = [plt.matplotlib.patches.Patch(facecolor=remaining_palette[i],
                                                    label=remaining_names[i]) for i in range(len(remaining_classes))]
    ax.legend(handles=legend_elements, title="Altered Classes", loc='upper left', bbox_to_anchor=(1.05, 1))
    plt.show()

    # Visualize SVM altered map on hillshade
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.imshow(hillshade_array, cmap='gray', extent=hillshade_extent, origin='upper') # Plot hillshade first
    im = ax.imshow(svm_altered_map, cmap=cmap_remaining, norm=norm_remaining, extent=extent, origin='upper', alpha=0.7) # Overlay altered map with transparency
    ax.set_title("SVM Altered Areas on Hillshade", fontsize=12)
    ax.set_xlabel("Longitude", fontsize=12)
    ax.set_ylabel("Latitude", fontsize=12)
    # Create a custom legend for the remaining classes
    legend_elements = [plt.matplotlib.patches.Patch(facecolor=remaining_palette[i],
                                                    label=remaining_names[i]) for i in range(len(remaining_classes))]
    ax.legend(handles=legend_elements, title="Altered Classes", loc='upper left', bbox_to_anchor=(1.05, 1))
    plt.show()

    # Visualize XGBoost altered map on hillshade
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.imshow(hillshade_array, cmap='gray', extent=hillshade_extent, origin='upper') # Plot hillshade first
    im = ax.imshow(xgb_altered_map, cmap=cmap_remaining, norm=norm_remaining, extent=extent, origin='upper', alpha=0.7) # Overlay altered map with transparency
    ax.set_title("XGBoost Altered Areas on Hillshade", fontsize=12)
    ax.set_xlabel("Longitude", fontsize=12)
    ax.set_ylabel("Latitude", fontsize=12)
    # Create a custom legend for the remaining classes
    legend_elements = [plt.matplotlib.patches.Patch(facecolor=remaining_palette[i],
                                                    label=remaining_names[i]) for i in range(len(remaining_classes))]
    ax.legend(handles=legend_elements, title="Altered Classes", loc='upper left', bbox_to_anchor=(1.05, 1))
    plt.show()

else:
    print("Cannot generate plots. Please check for errors in loading the hillshade or prediction maps.")

In [ ]:
import os

output_folder = '/content/drive/MyDrive/AGU'
os.makedirs(output_folder, exist_ok=True)

plt.savefig(f"{output_folder}/RF_Altered_Areas.png", dpi=1000, bbox_inches='tight')
plt.savefig(f"{output_folder}/SVM_Altered_Areas.png", dpi=1000, bbox_inches='tight')
plt.savefig(f"{output_folder}/XGBoost_Altered_Areas.png", dpi=1000, bbox_inches='tight')


In [ ]:
import os
import matplotlib.pyplot as plt

output_folder = '/content/drive/MyDrive/AGU'
os.makedirs(output_folder, exist_ok=True)

# Create figure with vertical layout
fig, axes = plt.subplots(3, 1, figsize=(12, 30), constrained_layout=True)

# RF Plot
axes[0].imshow(hillshade_array, cmap='gray', extent=hillshade_extent, origin='upper')
axes[0].imshow(rf_altered_map, cmap=cmap_remaining, norm=norm_remaining, extent=extent, origin='upper', alpha=0.7)
axes[0].set_title("Random Forest", fontsize=16)
axes[0].set_xlabel("Longitude")
axes[0].set_ylabel("Latitude")

# SVM Plot
axes[1].imshow(hillshade_array, cmap='gray', extent=hillshade_extent, origin='upper')
axes[1].imshow(svm_altered_map, cmap=cmap_remaining, norm=norm_remaining, extent=extent, origin='upper', alpha=0.7)
axes[1].set_title("SVM", fontsize=16)
axes[1].set_xlabel("Longitude")
axes[1].set_ylabel("Latitude")

# XGBoost Plot
axes[2].imshow(hillshade_array, cmap='gray', extent=hillshade_extent, origin='upper')
axes[2].imshow(xgb_altered_map, cmap=cmap_remaining, norm=norm_remaining, extent=extent, origin='upper', alpha=0.7)
axes[2].set_title("XGBoost", fontsize=16)
axes[2].set_xlabel("Longitude")
axes[2].set_ylabel("Latitude")

# Create a single legend below the last plot
legend_elements = [plt.matplotlib.patches.Patch(facecolor=remaining_palette[i],
                                                label=remaining_names[i]) for i in range(len(remaining_classes))]
fig.legend(handles=legend_elements, title="Altered Classes", loc='lower center',
           bbox_to_anchor=(0.5, -0.02), ncol=len(remaining_classes), fontsize=10, title_fontsize=12)

plt.subplots_adjust(hspace=0.05)  # Tight spacing between maps
plt.savefig(f"{output_folder}/RF_SVM_XGB_Combined_Vertical_Legend.png", dpi=1000, bbox_inches='tight')
plt.close(fig)


In [ ]:
from sklearn.ensemble import VotingClassifier

# Create an ensemble model using VotingClassifier
# We will use the best trained models
ensemble_model = VotingClassifier(
    estimators=[
        ('rf', best_rf),
        ('svm', best_svm),
        ('xgb', best_xgb)
    ],
    voting='hard'  # 'hard' voting uses predicted class labels
)

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Make predictions with the ensemble model
ensemble_preds = ensemble_model.predict(X_test)

# Evaluate the ensemble model
print("\n** Ensemble Model (Voting) **")
print("Accuracy:", (ensemble_preds == y_test).mean())

# Classification Report
print(classification_report(y_test, ensemble_preds, target_names=Names))

# Confusion Matrix
cm = confusion_matrix(y_test, ensemble_preds, labels=Classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=Classes)
disp.plot(cmap='Blues')
plt.title("Ensemble Model (Voting) - Confusion Matrix")
plt.show()

In [ ]:
# Make predictions on the entire image using the ensemble model
ensemble_prediction = ensemble_model.predict(img_reshaped)

# Reshape predictions back to image dimensions
ensemble_prediction_map = ensemble_prediction.reshape(height, width).astype(np.uint8)

# Calculate extent manually from transform and dimensions
# The transform matrix is a Rasterio Affine object.
# transform.c and transform.f are the x and y coordinates of the upper-left corner.
# transform.a and transform.e are the pixel width and height.
left = transform.c
top = transform.f
right = transform.c + transform.a * width
bottom = transform.f + transform.e * height
extent = [left, right, bottom, top]

# Prepare a discrete colormap
levels = Classes + [max(Classes) + 1]
cmap, norm = from_levels_and_colors(levels, Palette)

# Visualize Ensemble model prediction map
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(ensemble_prediction_map, cmap=cmap, norm=norm, extent=extent, origin='upper')
ax.set_title("Ensemble Model Classification Map", fontsize=12)
ax.set_xlabel("Longitude", fontsize=12)
ax.set_ylabel("Latitude", fontsize=12)
cbar = fig.colorbar(im, ax=ax, shrink=0.7, orientation='horizontal', pad=0.1) # Add pad for spacing
tick_positions = [i + 0.5 for i in Classes]
cbar.set_ticks(tick_positions)
cbar.set_ticklabels(Names)
plt.show()

In [ ]:
# Get feature importances from Random Forest
rf_feature_importances = best_rf.feature_importances_

# Create a DataFrame for visualization
rf_importance_df = pd.DataFrame({
    'Feature': Bands,
    'Importance': rf_feature_importances
})

# Sort by importance
rf_importance_df = rf_importance_df.sort_values('Importance', ascending=False)

# Plot Random Forest Feature Importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=rf_importance_df, palette='viridis')
plt.title('Random Forest Feature Importance')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

# Get feature importances from XGBoost
xgb_feature_importances = best_xgb.feature_importances_

# Create a DataFrame for visualization
xgb_importance_df = pd.DataFrame({
    'Feature': Bands,
    'Importance': xgb_feature_importances
})

# Sort by importance
xgb_importance_df = xgb_importance_df.sort_values('Importance', ascending=False)

# Plot XGBoost Feature Importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=xgb_importance_df, palette='viridis')
plt.title('XGBoost Feature Importance')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

                   CNN MODEL
                   

In [ ]:
# Install necessary libraries (if not installed)
!pip install rasterio scikit-learn matplotlib tensorflow
!pip install keras-tuner

In [ ]:
import pandas as pd
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
import os


In [ ]:
Sample_Path = '/content/drive/MyDrive/GEE_Exports/Training_Samples_Fixed.csv'
Image_Path = '/content/drive/MyDrive/GEE_Exports/ImageG_Stacked_Float.tif'

# Define target and predictor variables
Bands = ['PC2', 'PC3', 'PC5', 'geology','B09', 'B08', 'B07', 'B05', 'B06', 'B3N',
  'Calcite_SAM', 'Kaolinite_SAM', 'IronOxide_SAM']
Classes = [0, 1, 2, 3]
N_Classes = 4
Names = ["No_Alteration", "Calcite", "Kaolinite", "IronOxide"]
Palette = [
    '#008000',  # Green  for class 0 (No_Alteration)
    '#FF0000',  # Red    for class 1 (Calcite)
    '#0000FF',  # Blue   for class 2 (Kaolinite)
    '#FFD700',  # Gold   for class 3 (IronOxide)
]
Target = 'class'

# Encode target classes
le = LabelEncoder()
df['label'] = le.fit_transform(df[Target])

# Determine the number of unique classes
num_unique_classes = df['label'].nunique()

# Normalize input
scaler = MinMaxScaler()
X = scaler.fit_transform(df[Bands])
# Fix: Change num_classes to the actual number of unique classes
y = tf.keras.utils.to_categorical(df['label'], num_classes=num_unique_classes)

# Reshape for CNN
X = X.reshape(-1, 1, 1, len(Bands))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers

# Define the model building function for KerasTuner
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Conv2D(filters=hp.Int('filters', 32, 128, step=32),
                            kernel_size=(1, 1),
                            activation='relu',
                            input_shape=(1, 1, len(Bands))))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(filters=hp.Int('filters_2', 64, 256, step=64), # Add a second Conv2D layer
                            kernel_size=(1, 1),
                            activation='relu',
                            kernel_regularizer=regularizers.l2(hp.Float('l2_reg', 0.0001, 0.01, sampling='log')))) # Tune L2 regularization
    model.add(layers.BatchNormalization())
    model.add(layers.Flatten())
    model.add(layers.Dense(units=hp.Int('units', 32, 128, step=32), activation='relu'))
    model.add(layers.Dropout(rate=hp.Float('dropout', 0.2, 0.5, step=0.1)))
    model.add(layers.Dense(num_unique_classes, activation='softmax'))  # Use the determined number of unique classes

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('lr', [1e-3, 1e-4, 1e-5])), # Tune learning rate
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize tuner
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=10,  # Reduced trials for faster execution
                        executions_per_trial=1,
                        directory='alteration_tuning_cnn',
                        project_name='cnn_hyperparam_search')

# Run search
print("Starting hyperparameter search...")
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, verbose=1) # Increased epochs for better tuning

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

print("\nEvaluating the best model on the test data...")
loss, accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

y_pred = best_model.predict(X_test)

In [ ]:
!pip install keras-tuner

In [ ]:
def create_model():
    model = models.Sequential([
        layers.Conv2D(64, (1, 1), activation='relu', input_shape=(1, 1, len(Bands))),
        layers.BatchNormalization(),
        layers.Conv2D(128, (1, 1), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(num_unique_classes, activation='softmax')  # Use the determined number of unique classes
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_model()

# Train
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=16,
    verbose=1
)

In [ ]:
save_path = '/content/drive/MyDrive/PCalcite'

# Accuracy plot
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('CNN Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig(f'{save_path}/cnn_accuracy.png', dpi=1000)
plt.show()

# Loss plot
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('CNN Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig(f'{save_path}/cnn_loss.png', dpi=1000)
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_pred_labels = np.argmax(y_pred, axis=1)

# Confusion matrix
cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_,
            yticklabels=le.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Load the stacked raster
with rasterio.open(Image_Path) as src:
    img = src.read().astype('float32')
    profile = src.profile

# Normalize the image using same scaler
img_reshaped = img.reshape(img.shape[0], -1).T
img_scaled = scaler.transform(img_reshaped)
img_scaled = img_scaled.reshape(-1, 1, 1, len(Bands))

# Predict class for each pixel
predictions = model.predict(img_scaled)
predicted_classes = np.argmax(predictions, axis=1)
pred_map = predicted_classes.reshape(profile['height'], profile['width'])

# Export classified image
out_path = f"{save_path}/Predicted_Map_CNN.tif"
profile.update(dtype=rasterio.uint8, count=1)
with rasterio.open(out_path, 'w', **profile) as dst:
    dst.write(pred_map.astype(rasterio.uint8), 1)

# Plot
plt.figure(figsize=(10, 8))
plt.imshow(pred_map, cmap='viridis')
plt.title("CNN")
plt.axis('off')
plt.savefig(f'{save_path}/Predicted_Map_CNN.png', dpi=1000)
plt.show()

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import from_levels_and_colors
from matplotlib.patches import Patch

# Define the path to the hillshade image
Hillshade_Path = '/content/drive/MyDrive/GEE_Exports/Hillshade.tif'

# Load the hillshade image
try:
    with rasterio.open(Hillshade_Path) as hillshade_image:
        hillshade_array = hillshade_image.read(1) # Read the first band (assuming it's a single-band hillshade)
        hillshade_extent = [hillshade_image.bounds.left, hillshade_image.bounds.right,
                          hillshade_image.bounds.bottom, hillshade_image.bounds.top]
except rasterio.errors.RasterioIOError:
    print(f"Error: Could not open or read the hillshade image at {Hillshade_Path}")
    hillshade_array = None

# Ensure the extent of the classification maps matches the hillshade for overlay
# Use the extent calculated from the classification image's transform
# (Assuming the classification image and hillshade have the same extent and resolution)
# If not, resampling or reprojecting might be necessary.
# For this code, we will assume the extents align.
if 'extent' not in locals(): # Check if extent from classification map loading is defined
    # If not defined, calculate it from the loaded classification image (from cell RP8BRkZknU8n)
    # Need access to 'transform', 'height', 'width' from previous cells
    try:
        bbox = rasterio.coords.BoundingBox(
            left=transform.c,
            bottom=transform.f + transform.e * height,
            right=transform.c + transform.a * width,
            top=transform.f
        )
        extent = [bbox.left, bbox.right, bbox.bottom, bbox.top]
    except NameError:
        print("Error: 'transform', 'height', or 'width' not defined. Cannot calculate extent.")
        extent = None


# Reshape predictions back to image dimensions (ensure these variables are available)
# ensemble_prediction and predicted_classes (for CNN) should be available from previous cells
try:
    if 'ensemble_prediction' not in locals():
        print("Error: 'ensemble_prediction' is not defined. Please run the cell that generates ensemble predictions.")
        ensemble_prediction_map = None
    else:
        ensemble_prediction_map = ensemble_prediction.reshape(height, width).astype(np.uint8)

    if 'predicted_classes' not in locals():
        print("Error: 'predicted_classes' (CNN) is not defined. Please run the cell that generates CNN predictions.")
        cnn_prediction_map = None
    else:
        cnn_prediction_map = predicted_classes.reshape(height, width).astype(np.uint8)

except NameError:
    print("Error: 'height' or 'width' not defined. Cannot reshape prediction maps.")
    ensemble_prediction_map = None
    cnn_prediction_map = None


if hillshade_array is not None and extent is not None:
    # Prepare a discrete colormap (excluding No_Alteration class 0)
    # We need one more level than classes for from_levels_and_colors for the remaining classes
    remaining_classes = [c for c in Classes if c != 0]
    remaining_names = [Names[i] for i in remaining_classes]
    remaining_palette = [Palette[i] for i in remaining_classes]

    # Adjust levels and colormap for the remaining classes
    # Levels should span from the minimum remaining class value to max remaining class value + 1
    levels_remaining = remaining_classes + [max(remaining_classes) + 1]
    cmap_remaining, norm_remaining = from_levels_and_colors(levels_remaining, remaining_palette)


    # --- Masking and Plotting ---

    if ensemble_prediction_map is not None:
        # Create mask for the 'No_Alteration' class (class 0) for Ensemble
        mask_ensemble_no_alteration = (ensemble_prediction_map == 0)

        # Apply mask to keep only the altered classes, setting masked values to NaN
        ensemble_altered_map = ensemble_prediction_map.astype(float) # Convert to float to allow NaN
        ensemble_altered_map[mask_ensemble_no_alteration] = np.nan

        # Visualize Ensemble altered map on hillshade using the provided template
        fig, ax = plt.subplots(figsize=(10, 8))
        ax.imshow(hillshade_array, cmap='gray', extent=hillshade_extent, origin='upper') # Plot hillshade first
        im = ax.imshow(ensemble_altered_map, cmap=cmap_remaining, norm=norm_remaining, extent=extent, origin='upper', alpha=0.7) # Overlay altered map with transparency
        ax.set_title("Ensemble Model Altered Areas on Hillshade", fontsize=12)
        ax.set_xlabel("Longitude", fontsize=12)
        ax.set_ylabel("Latitude", fontsize=12)

        # Create a custom legend for the remaining classes
        legend_elements = [Patch(facecolor=remaining_palette[i],
                                                        label=remaining_names[i]) for i in range(len(remaining_classes))]
        ax.legend(handles=legend_elements, title="Altered Classes", loc='upper left', bbox_to_anchor=(1.05, 1))

        plt.show()

    if cnn_prediction_map is not None:
         # Create mask for the 'No_Alteration' class (class 0) for CNN
        mask_cnn_no_alteration = (cnn_prediction_map == 0)

        # Apply mask to keep only the altered classes, setting masked values to NaN
        cnn_altered_map = cnn_prediction_map.astype(float) # Convert to float to allow NaN
        cnn_altered_map[mask_cnn_no_alteration] = np.nan

        # Visualize CNN altered map on hillshade using the provided template
        fig, ax = plt.subplots(figsize=(10, 8))
        ax.imshow(hillshade_array, cmap='gray', extent=hillshade_extent, origin='upper') # Plot hillshade first
        im = ax.imshow(cnn_altered_map, cmap=cmap_remaining, norm=norm_remaining, extent=extent, origin='upper', alpha=0.7) # Overlay altered map with transparency
        ax.set_title("CNN Altered Areas on Hillshade", fontsize=12)
        ax.set_xlabel("Longitude", fontsize=12)
        ax.set_ylabel("Latitude", fontsize=12)

        # Create a custom legend for the remaining classes
        legend_elements = [Patch(facecolor=remaining_palette[i],
                                                        label=remaining_names[i]) for i in range(len(remaining_classes))]
        ax.legend(handles=legend_elements, title="Altered Classes", loc='upper left', bbox_to_anchor=(1.05, 1))

        plt.show()


elif hillshade_array is None:
    print("Cannot generate plots because the hillshade image could not be loaded.")
else:
     print("Cannot generate plots because prediction maps or extent could not be determined.")